# TDC Calibration

## define the diagnostic

In [ ]:
from plugins.interfaces.diagnostics import EPICSImageDiagnostic, ROI

screen_name = "13ARV1"
save_image_location = ""
dyg14_roi = ROI(
    ymin=280, ymax=1000, xmin=480, xmax=1150
)
charge_pvs = []

image_diagnostic = EPICSImageDiagnostic(
    screen_name=screen_name,
    resolution_suffix=None,
    roi=dyg14_roi,
    extra_pvs=charge_pvs,
    save_image_location=save_image_location
)

In [ ]:
image_diagnostic.test_measurement()

### Define the evaluator

In [1]:
from epics import caput, caget
from xopt import Evaluator
import time
import numpy as np

def evaluate_function(inputs: dict) -> dict:
    global image_diagnostic
    # caput values
    for name, val in inputs:
        caput(name, val)

    # wait for changes to occur - use small wait time for interpolated measurements
    time.sleep(1)

    results = image_diagnostic.measure_beamsize(5, **inputs)

    # get other PV's NOTE: Measurements not synchronous with beamsize measurements!
    results = results

    # add total beam size
    results["total_size"] = np.sqrt(np.array(results["Sx"]) ** 2 + np.array(results["Sy"]) ** 2)
    return results

evaluator = Evaluator(function=evaluate_function)

### Define VOCS
Here we define the names and ranges of input parameters, the names and settings of
objectives, and the names and settings of constraints. Note that the keys here should
 be referenced in the evaluate function above.

In [2]:
from xopt import VOCS

TDC_PHASE_PV = ""
# define control PVs and ranges here
vocs = VOCS(
    variables = {
        TDC_PHASE_PV: [0, 1], # Quad 4:9
    },
    objectives = {"total_size":"MINIMIZE"},
)

### Define the Generator (NOT USED HERE)

In [3]:
from xopt.generators import get_generator

generator = get_generator("random")(vocs=vocs)


    Random number generator.
    


###  Combine into Xopt object

In [10]:
from xopt import Xopt
dump_filename = "tdc_calibration.yml"
X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator, dump_file=dump_filename)

## Introspection
Objects in Xopt can be printed to a string or dumped to a text file for easy
introspection of attributes and current configuration.

In [11]:
# Convenient representation of the state.
X


            Xopt
________________________________
Version: 2.1.0+5.g375e990d
Data size: 0
Config as YAML:
dump_file: my_data.yml
evaluator:
  function: __main__.evaluate_function
  function_kwargs: {}
  max_workers: 1
  vectorized: false
generator:
  name: random
  supports_batch_generation: true
  supports_multi_objective: true
max_evaluations: null
serialize_inline: false
serialize_torch: false
strict: true
vocs:
  constants: {}
  constraints: {}
  objectives: {}
  observables:
  - my_pv
  variables:
    x1:
    - 0.0
    - 1.0
    x2:
    - 0.0
    - 1.0
    x3:
    - 0.0
    - 1.0


### Do scan

In [ ]:
# examine the data stored in Xopt
tdc_phase_values = np.linspace(0,1,10)
X.evaluate_data({TDC_PHASE_PV: tdc_phase_values})

In [ ]:
# visualize result
X.data.plot(y="CY")

### Polynomial fit for TDC calibration

In [ ]:
# do a linear fit 
import numpy as np
p = np.polyfit(X.data[TDC_PHASE_PV], X.data["CY"], 1)
p
